<a href="https://colab.research.google.com/github/anurvish/Scalable_Data/blob/main/DummyRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize,StandardScaler,MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error,r2_score

In [20]:
#reading the data from CSV file
df = pd.read_csv("/content/drive/MyDrive/ML/GroupProject/final_data.csv", )
df.head()
#Obtained data preprocessing
df = df.replace(np.NaN, 0)
df.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
df.drop(labels=['release_date_main'],axis=1,inplace=True)
df.drop(labels=['id_main'],axis=1,inplace=True)
df.drop(labels=['runtime_main'],axis=1,inplace=True)
Independent_features = df.iloc[:,1:]
Target = df.iloc[:,0]
Independent_features = Independent_features.replace(np.nan,0)

In [21]:
# calculate Variance inflation factor for the given data frame
def calculate_vif(df, features):    
    vif, tolerance = {}, {}
    # all the features that you want to examine
    for feature in features:
        # extract all the other features you will regress against
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        # extract r-squared from the fitb
        r2 = LinearRegression().fit(X, y).score(X, y)                
        
        # calculate tolerance
        tolerance[feature] = 1 - r2
        # calculate VIF
        vif[feature] = 1/(tolerance[feature])
    # return VIF DataFrame
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

In [22]:
#setting the max rows 
pd.set_option('max_rows', 99999)
df.head(5)

,revenue_main,adult_main,budget_main,Action_genre_main,Adventure_genre_main,Animation_genre_main,Comedy_genre_main,Crime_genre_main,Documentary_genre_main,Drama_genre_main,Family_genre_main,Fantasy_genre_main,History_genre_main,Horror_genre_main,Music_genre_main,Mystery_genre_main,Romance_genre_main,Science Fiction_genre_main,TV Movie_genre_main,Thriller_genre_main,War_genre_main,Western_genre_main,collection_popularity_coll_details,popularity_crew,0_gender_crew,1_gender_crew,2_gender_crew,3_gender_crew,Directing_relevance_crew,popularity_crew_2,0_gender_crew_2,1_gender_crew_2,2_gender_crew_2,3_gender_crew_2,Directing_relevance_crew_2,popularity_crew_3,0_gender_crew_3,1_gender_crew_3,2_gender_crew_3,3_gender_crew_3,Directing_relevance_crew_3,popularity_cast,2_gender_cast,1_gender_cast,0_gender_cast,3_gender_cast,Acting_relevance_cast,popularity_cast_2,2_gender_cast_2,1_gender_cast_2,0_gender_cast_2,3_gender_cast_2,Acting_relevance_cast_2,popularity_cast_3,2_gender_cast_3,1_gender_cast_3,0_gender_cast_3,3_gender_cast_3,Acting_relevance_cast_3,popularity_cast_4,2_gender_cast_4,1_gender_cast_4,0_gender_cast_4,3_gender_cast_4,Acting_relevance_cast_4,popularity_cast_5,2_gender_cast_5,1_gender_cast_5,0_gender_cast_5,3_gender_cast_5,Acting_relevance_cast_5
0,5303,False,130178,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,33.365000,0.631,0,0,1,0,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0.600,1,0,0,0,1,0.600,1,0,0,0,1,0.000,0,0,0,0,0,0.600,0,1,0,0,1,1.452,1,0,0,0,1
1,7000000,False,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,27.062667,1.967,0,0,1,0,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,8.085,0,1,0,0,1,3.798,0,1,0,0,1,4.907,1,0,0,0,1,1.626,1,0,0,0,1,0.000,0,0,0,0,0
2,42000000,False,5000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0.000000,0.751,0,0,1,0,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,5.795,1,0,0,0,1,1.638,0,1,0,0,1,2.547,1,0,0,0,1,0.872,1,0,0,0,1,0.000,0,0,0,0,0
3,8900000,False,7000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,13.355000,0.000,0,0,0,0,0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,5.865,0,1,0,0,1,3.550,1,0,0,0,1,2.203,1,0,0,0,1,1.879,1,0,0,0,1,0.000,0,0,0,0,0
4,127345,False,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,5.375667,0.694,0,0,1,0,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,5.881,1,0,0,0,1,1.400,0,1,0,0,1,1.166,0,1,0,0,1,0.732,0,1,0,0,1,0.000,0,0,0,0,0


In [ ]:
df_vif = calculate_vif(Independent_features,['budget_main','Action_genre_main',
       'Adventure_genre_main', 'Animation_genre_main', 'Comedy_genre_main',
       'Crime_genre_main', 'Documentary_genre_main', 'Drama_genre_main',
       'Family_genre_main', 'Fantasy_genre_main', 'History_genre_main',
       'Horror_genre_main', 'Music_genre_main', 'Mystery_genre_main',
       'Romance_genre_main', 'Science Fiction_genre_main',
       'TV Movie_genre_main', 'Thriller_genre_main', 'War_genre_main',
       'Western_genre_main', 'collection_popularity_coll_details','popularity_crew', '0_gender_crew', '1_gender_crew',
       '2_gender_crew','Directing_relevance_crew', 'popularity_crew_2',
       '0_gender_crew_2', '1_gender_crew_2', '2_gender_crew_2','Directing_relevance_crew_2', 'popularity_crew_3',
       '0_gender_crew_3', '1_gender_crew_3', '2_gender_crew_3','Directing_relevance_crew_3', 'popularity_cast',
       '2_gender_cast', '1_gender_cast', '0_gender_cast', '3_gender_cast',
       'Acting_relevance_cast', 'popularity_cast_2', '2_gender_cast_2',
       '1_gender_cast_2', '0_gender_cast_2', '3_gender_cast_2',
       'Acting_relevance_cast_2', 'popularity_cast_3', '2_gender_cast_3',
       '1_gender_cast_3', '0_gender_cast_3', '3_gender_cast_3',
       'Acting_relevance_cast_3', 'popularity_cast_4', '2_gender_cast_4',
       '1_gender_cast_4', '0_gender_cast_4', '3_gender_cast_4',
       'Acting_relevance_cast_4', 'popularity_cast_5', '2_gender_cast_5',
       '1_gender_cast_5', '0_gender_cast_5', '3_gender_cast_5',
       'Acting_relevance_cast_5'])
df_vif

In [24]:
Independent_features = Independent_features[['budget_main','Action_genre_main',
       'Adventure_genre_main', 'Animation_genre_main', 'Comedy_genre_main',
       'Crime_genre_main', 'Documentary_genre_main', 'Drama_genre_main',
       'Family_genre_main', 'Fantasy_genre_main', 'History_genre_main',
       'Horror_genre_main', 'Music_genre_main', 'Mystery_genre_main',
       'Romance_genre_main', 'Science Fiction_genre_main',
       'TV Movie_genre_main', 'Thriller_genre_main', 'War_genre_main',
       'Western_genre_main', 'collection_popularity_coll_details','popularity_crew', 'popularity_crew_2', 'popularity_crew_3', 'popularity_cast', 
       'popularity_cast_2', 'popularity_cast_3', 'popularity_cast_4', 'popularity_cast_5']]

In [25]:
mms_budget = MinMaxScaler()
mms_budget.fit(Independent_features['budget_main'].to_numpy().reshape(-1,1))

mms_revenue = MinMaxScaler()
mms_revenue.fit(Target.values.reshape(-1,1))

MinMaxScaler()

In [26]:
Independent_features['budget_main'] = mms_budget.fit_transform(Independent_features['budget_main'].to_numpy().reshape(-1,1))
Target = mms_revenue.fit_transform(Target.values.reshape(-1,1))

In [27]:
#splitting the final data into train and test data 
xtrain, xtest, ytrain, ytest = train_test_split(Independent_features, Target, test_size=0.2)

In [30]:
dummy = DummyRegressor(strategy='quantile',quantile=0.25)
dummy.fit(xtrain,ytrain)
testDummyPred = dummy.predict(xtest)
trainDummyPred = dummy.predict(xtrain)
print(f'Test r2(R_Square): {r2_score(ytest,testDummyPred)} ')
print(f'Train r2(R_Square): {r2_score(ytrain,trainDummyPred)} ')
print(f'Test Mean squared error: {mean_squared_error(ytest, testDummyPred)} & Mean Absulute error: {mean_absolute_error(ytest, testDummyPred)}')
print(f'Train Mean squared error: {mean_squared_error(ytrain, trainDummyPred)} & Mean Absulute error: {mean_absolute_error(ytrain, trainDummyPred)}')
#RMSE
print(f'Test RMSE :{np.square(np.subtract(ytest,testDummyPred)).mean()}')
print(f'Train RMSE: {np.square(np.subtract(ytrain,trainDummyPred)).mean()}')
print("\n\n")

Test r2(R_Square): -0.21718540567895261 
Train r2(R_Square): -0.19757658986898963 
Test Mean squared error: 0.0038452246429761584 & Mean Absulute error: 0.026490090429671494
Train Mean squared error: 0.003860934235147904 & Mean Absulute error: 0.02556249222490498
Test RMSE :0.0038452246429761558
Train RMSE: 0.003860934235147906



